# Day 1 실습 6: 2개 태스크로 빠른 LoRA 파인튜닝

## 🎯 이 노트북의 목적

**동일한 모델(EXAONE-3.5)**을 **2가지 다른 태스크**로 파인튜닝하여 비교합니다.

| 태스크 | 데이터셋 | 목표 | 샘플 수 |
|--------|----------|------|---------|
| **감정 분류** | IMDB | 영화 리뷰 → 긍정/부정 | 100 |
| **영어 QA** | SQuAD | 지문 + 질문 → 답변 | 100 |

**한국어 요약**은 03번에서 이미 학습했으므로 제외합니다.

### 학습 목표

- ✅ **Task별 LoRA 학습**: 같은 모델도 데이터가 다르면 다르게 학습됨
- ✅ **빠른 실험**: 각 200개씩만 사용해 빠르게 비교
- ✅ **성능 비교**: 어떤 태스크가 쉽고 어려운지 확인

### 실습 흐름

1. 2개 태스크 데이터 준비
2. 각 태스크별로 LoRA 학습 (동일 설정: r=8)
3. Baseline vs LoRA 비교 평가
4. 결과 비교 및 인사이트

## 1. 패키지 설치

Colab이라면 아래를 실행하고 **런타임 재시작**하세요.

In [ ]:
!pip install -q --upgrade bitsandbytes accelerate peft datasets transformers evaluate

# 설치 후 런타임 재시작 필요!

## 2. 라이브러리 로드

In [ ]:
import os
import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import evaluate

print("✅ 라이브러리 로드 완료")

## 3. 태스크 설정

각 태스크의 데이터셋과 프롬프트 템플릿을 정의합니다.

In [ ]:
# 태스크 설정 (2개만)
TASKS = {
    "sentiment": {
        "name": "감정 분류",
        "dataset": "imdb",
        "split": "train[:100]",
        "eval_split": "test[:10]",
        "input_col": "text",
        "output_col": "label",
        "prompt_template": "다음 영화 리뷰의 감정을 분류하세요.\n\n리뷰: {input}\n\n감정:",
        "label_map": {0: "부정", 1: "긍정"},
        "description": "영화 리뷰 감정을 긍정/부정으로 분류"
    },
    "qa": {
        "name": "영어 QA",
        "dataset": "squad",
        "split": "train[:100]",
        "eval_split": "validation[:10]",
        "input_col": ["question", "context"],
        "output_col": "answers",
        "prompt_template": "Context: {context}\n\nQuestion: {question}\n\nAnswer:",
        "description": "영어 지문과 질문을 받아 답변 생성"
    }
}

# 공통 모델 설정
BASE_MODEL = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

# LoRA 설정 (모든 태스크 동일)
LORA_CONFIG = {
    "r": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.05,
    "target_modules": ["q_proj", "k_proj", "v_proj", "out_proj"],
}

# 학습 설정 (모든 태스크 동일)
TRAINING_CONFIG = {
    "num_train_epochs": 3,
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-4,
    "max_steps": 50,  # 빠른 테스트용
}

print("✅ 태스크 설정 완료\n")
for task_id, task in TASKS.items():
    print(f"📌 {task['name']}: {task['description']}")

# 토크나이저 미리 로드 (전역에서 한 번만)
print("📥 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
    padding_side="right",
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ 토크나이저 로드 완료")


## 4. 데이터 로드 및 전처리 함수

In [ ]:
def load_task_data(task_id):
    """태스크별 데이터 로드 및 프롬프트 생성"""
    task = TASKS[task_id]
    
    print(f"\n📥 {task['name']} 데이터 로드 중...")
    
    # 데이터셋 로드
    dataset = load_dataset(task["dataset"], split=task["split"])
    
    # 프롬프트 생성
    processed_data = []
    
    for item in dataset:
        # Input 처리
        if isinstance(task["input_col"], list):
            # 여러 컬럼 (QA)
            inputs = {col: str(item[col]) for col in task["input_col"]}
            prompt = task["prompt_template"].format(**inputs)
        else:
            # 단일 컬럼
            input_text = str(item[task["input_col"]])
            prompt = task["prompt_template"].format(input=input_text)
        
        # Output 처리
        output_col = task["output_col"]
        if output_col == "label":
            # 감정 분류
            label = item[output_col]
            output = task["label_map"][label]
        elif output_col == "answers":
            # SQuAD
            answers = item[output_col]["text"]
            output = answers[0] if answers else ""
        else:
            # 일반 텍스트
            output = str(item[output_col])
        
        processed_data.append({
            "text": prompt,
            "answer": output
        })
    
    df = pd.DataFrame(processed_data)
    print(f"   ✅ {len(df)}개 샘플 준비")
    
    return df

print("✅ 데이터 로드 함수 정의 완료")

## 5. 공통 학습 함수

In [ ]:
def train_lora_for_task(task_id, output_dir):
    """태스크별 LoRA 학습 실행"""
    task = TASKS[task_id]
    
    print(f"\n{'='*60}")
    print(f"🚀 {task['name']} 학습 시작")
    print(f"{'='*60}\n")
    
    # 1. 데이터 로드
    train_df = load_task_data(task_id)
    
    # 2. 모델 로드 (4-bit quantization)
    print("\n📥 모델 로드 중...")
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    
    # 3. LoRA 설정
    lora_config = LoraConfig(
        task_type="CAUSAL_LM",
        r=LORA_CONFIG["r"],
        lora_alpha=LORA_CONFIG["lora_alpha"],
        lora_dropout=LORA_CONFIG["lora_dropout"],
        target_modules=LORA_CONFIG["target_modules"],
        bias="none",
    )
    
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    
    print(f"   ✅ LoRA 적용 완료 (r={LORA_CONFIG['r']})")
    
    # 4. 데이터 전처리
    def prepare_example(row):
        prompt = row["text"]
        answer = row["answer"]
        full_text = prompt + " " + answer + tokenizer.eos_token
        
        # 프롬프트만 토큰화 (라벨 마스크용)
        prompt_tokens = tokenizer(
            prompt,
            add_special_tokens=False,
            truncation=True,
            max_length=512,
        )
        
        # 전체 토큰화
        full_tokens = tokenizer(
            full_text,
            truncation=True,
            max_length=512,
            padding="max_length",
        )
        
        input_ids = full_tokens["input_ids"]
        attention_mask = full_tokens["attention_mask"]
        labels = input_ids.copy()
        
        # 프롬프트 부분 마스킹
        prompt_len = len(prompt_tokens["input_ids"])
        labels[:prompt_len] = [-100] * prompt_len
        
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }
    
    from datasets import Dataset
    train_dataset = Dataset.from_pandas(train_df)
    train_dataset = train_dataset.map(lambda x: prepare_example(x))
    
    # 5. 학습 실행
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=TRAINING_CONFIG["num_train_epochs"],
        per_device_train_batch_size=TRAINING_CONFIG["per_device_train_batch_size"],
        gradient_accumulation_steps=TRAINING_CONFIG["gradient_accumulation_steps"],
        learning_rate=TRAINING_CONFIG["learning_rate"],
        max_steps=TRAINING_CONFIG["max_steps"],
        logging_steps=10,
        save_strategy="no",
        report_to=[],
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )
    
    print("\n🏋️ 학습 중...")
    trainer.train()
    
    # 6. 저장
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    print(f"\n✅ 학습 완료! 저장 경로: {output_dir}")
    
    return model, tokenizer

print("✅ 학습 함수 정의 완료")

## 6. Task 1: 감정 분류 학습

영화 리뷰를 긍정/부정으로 분류하는 태스크입니다.

In [ ]:
# Task 1: 감정 분류
task1_dir = "./lora_sentiment"

model_sent, tokenizer_sent = train_lora_for_task("sentiment", task1_dir)

print("\n📊 Task 1 완료!")

## 7. Task 2: 영어 QA 학습

영어 지문과 질문을 받아 답변을 생성하는 태스크입니다.

In [ ]:
# Task 2: 영어 QA
task2_dir = "./lora_qa"

model_qa, tokenizer_qa = train_lora_for_task("qa", task2_dir)

print("\n📊 Task 2 완료!")

## 8. 학습 결과 요약

2개 태스크 모두 학습이 완료되었습니다!

각 태스크별로 LoRA 어댑터가 저장되었습니다:
- `./lora_sentiment` - 감정 분류
- `./lora_qa` - 영어 QA

In [ ]:
results_summary = pd.DataFrame([
    {
        "Task": "감정 분류",
        "Dataset": "IMDB",
        "Samples": 100,
        "LoRA Path": "./lora_sentiment",
        "Description": "리뷰 → 긍정/부정"
    },
    {
        "Task": "영어 QA",
        "Dataset": "SQuAD",
        "Samples": 100,
        "LoRA Path": "./lora_qa",
        "Description": "지문 + 질문 → 답변"
    }
])

print("\n" + "="*80)
print("📊 학습 결과 요약")
print("="*80 + "\n")
display(results_summary)

## 9. 평가 도구 로드

ROUGE-1과 임베딩 유사도 평가를 위한 도구를 준비합니다.

## 10. Baseline vs LoRA 평가

각 태스크별로 **파인튜닝 전(Baseline)**과 **파인튜닝 후(LoRA)**를 비교합니다.

In [ ]:
!pip install rouge_score

In [ ]:
from sentence_transformers import SentenceTransformer
import evaluate
import numpy as np

print("📥 평가 도구 로드 중...")
rouge = evaluate.load("rouge")
embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda" if torch.cuda.is_available() else "cpu")

def compute_rouge1(reference, candidate):
    try:
        return rouge.compute(predictions=[candidate], references=[reference])["rouge1"]
    except:
        return 0.0

def compute_embedding_similarity(reference, candidate):
    try:
        if not reference.strip() or not candidate.strip():
            return 0.0
        embeddings = embed_model.encode([reference, candidate], convert_to_numpy=True, normalize_embeddings=True)
        return float(np.dot(embeddings[0], embeddings[1]))
    except:
        return 0.0

print("✅ 평가 도구 준비 완료")

In [ ]:
def evaluate_task(task_id, lora_model, tokenizer, num_samples=3):
    """태스크별 Baseline vs LoRA 평가"""
    task = TASKS[task_id]
    
    print(f"\n{'='*80}")
    print(f"🔍 {task['name']} 평가 중...")
    print(f"{'='*80}\n")
    
    # 1. 평가 데이터 로드
    eval_dataset = load_dataset(task["dataset"], split=task["eval_split"])
    eval_samples = eval_dataset.select(range(min(num_samples, len(eval_dataset))))
    
    # 2. Baseline 모델 로드
    print("📥 Baseline 모델 로드 중...")
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    )
    
    baseline_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    
    # 3. 각 샘플 평가
    results = []
    
    for idx, item in enumerate(eval_samples):
        # Input 처리
        if isinstance(task["input_col"], list):
            inputs = {col: str(item[col]) for col in task["input_col"]}
            prompt = task["prompt_template"].format(**inputs)
            input_display = inputs["question"][:50] + "..."
        else:
            input_text = str(item[task["input_col"]])
            prompt = task["prompt_template"].format(input=input_text)
            input_display = input_text[:50] + "..."
        
        # Reference 처리
        output_col = task["output_col"]
        if output_col == "label":
            label = item[output_col]
            reference = task["label_map"][label]
        elif output_col == "answers":
            answers = item[output_col]["text"]
            reference = answers[0] if answers else ""
        else:
            reference = str(item[output_col])
        
        # Baseline 생성
        inputs_tokens = tokenizer(prompt, return_tensors="pt").to(baseline_model.device)
        with torch.no_grad():
            baseline_outputs = baseline_model.generate(
                **inputs_tokens,
                max_new_tokens=60 if task_id == "qa" else 10,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        baseline_result = tokenizer.decode(baseline_outputs[0], skip_special_tokens=True)
        
        # 프롬프트 제거
        if "감정:" in baseline_result:
            baseline_output = baseline_result.split("감정:")[-1].strip()
        elif "Answer:" in baseline_result:
            baseline_output = baseline_result.split("Answer:")[-1].strip()
        else:
            baseline_output = baseline_result[len(prompt):].strip()
        
        # LoRA 생성
        inputs_tokens = tokenizer(prompt, return_tensors="pt").to(lora_model.device)
        with torch.no_grad():
            lora_outputs = lora_model.generate(
                **inputs_tokens,
                max_new_tokens=60 if task_id == "qa" else 10,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        lora_result = tokenizer.decode(lora_outputs[0], skip_special_tokens=True)
        
        # 프롬프트 제거
        if "감정:" in lora_result:
            lora_output = lora_result.split("감정:")[-1].strip()
        elif "Answer:" in lora_result:
            lora_output = lora_result.split("Answer:")[-1].strip()
        else:
            lora_output = lora_result[len(prompt):].strip()
        
        # 점수 계산
        baseline_rouge = compute_rouge1(reference, baseline_output)
        lora_rouge = compute_rouge1(reference, lora_output)
        baseline_emb = compute_embedding_similarity(reference, baseline_output)
        lora_emb = compute_embedding_similarity(reference, lora_output)
        
        results.append({
            "input": input_display,
            "reference": reference[:100],
            "baseline": baseline_output[:100],
            "lora": lora_output[:100],
            "baseline_rouge1": baseline_rouge,
            "lora_rouge1": lora_rouge,
            "baseline_emb": baseline_emb,
            "lora_emb": lora_emb,
        })
        
        print(f"   {idx + 1}/{num_samples} 완료")
    
    # DataFrame 생성
    results_df = pd.DataFrame(results).round(3)
    
    # 평균 계산
    avg_results = {
        "task": task["name"],
        "baseline_rouge1": results_df["baseline_rouge1"].mean(),
        "lora_rouge1": results_df["lora_rouge1"].mean(),
        "baseline_emb": results_df["baseline_emb"].mean(),
        "lora_emb": results_df["lora_emb"].mean(),
        "rouge_improvement": results_df["lora_rouge1"].mean() - results_df["baseline_rouge1"].mean(),
        "emb_improvement": results_df["lora_emb"].mean() - results_df["baseline_emb"].mean(),
    }
    
    print(f"\n✅ {task['name']} 평가 완료!")
    
    return results_df, avg_results

print("✅ 평가 함수 정의 완료")

## 11. Task 1 평가: 감정 분류

In [ ]:
# Task 1: 감정 분류 평가
results_df_sent, avg_sent = evaluate_task("sentiment", model_sent, tokenizer_sent, num_samples=3)

print("\n" + "="*80)
print("📊 감정 분류 - 상세 결과")
print("="*80)
display(results_df_sent)

print(f"\n평균 ROUGE-1: Baseline {avg_sent['baseline_rouge1']:.3f} → LoRA {avg_sent['lora_rouge1']:.3f} (개선: {avg_sent['rouge_improvement']:+.3f})")
print(f"평균 임베딩:  Baseline {avg_sent['baseline_emb']:.3f} → LoRA {avg_sent['lora_emb']:.3f} (개선: {avg_sent['emb_improvement']:+.3f})")

## 12. Task 2 평가: 영어 QA

In [ ]:
# Task 2: 영어 QA 평가
results_df_qa, avg_qa = evaluate_task("qa", model_qa, tokenizer_qa, num_samples=3)

print("\n" + "="*80)
print("📊 영어 QA - 상세 결과")
print("="*80)
display(results_df_qa)

print(f"\n평균 ROUGE-1: Baseline {avg_qa['baseline_rouge1']:.3f} → LoRA {avg_qa['lora_rouge1']:.3f} (개선: {avg_qa['rouge_improvement']:+.3f})")
print(f"평균 임베딩:  Baseline {avg_qa['baseline_emb']:.3f} → LoRA {avg_qa['lora_emb']:.3f} (개선: {avg_qa['emb_improvement']:+.3f})")

## 13. 전체 비교 요약

2개 태스크의 성능을 한눈에 비교합니다.

In [ ]:
# 전체 비교표
comparison_df = pd.DataFrame([
    {
        "Task": avg_sent["task"],
        "Baseline ROUGE": avg_sent["baseline_rouge1"],
        "LoRA ROUGE": avg_sent["lora_rouge1"],
        "ROUGE 개선": avg_sent["rouge_improvement"],
        "Baseline 임베딩": avg_sent["baseline_emb"],
        "LoRA 임베딩": avg_sent["lora_emb"],
        "임베딩 개선": avg_sent["emb_improvement"],
    },
    {
        "Task": avg_qa["task"],
        "Baseline ROUGE": avg_qa["baseline_rouge1"],
        "LoRA ROUGE": avg_qa["lora_rouge1"],
        "ROUGE 개선": avg_qa["rouge_improvement"],
        "Baseline 임베딩": avg_qa["baseline_emb"],
        "LoRA 임베딩": avg_qa["lora_emb"],
        "임베딩 개선": avg_qa["emb_improvement"],
    }
]).round(3)

print("\n" + "="*80)
print("📊 2개 태스크 종합 비교")
print("="*80 + "\n")
display(comparison_df)

# 시각화
print("\n개선도 요약:")
for _, row in comparison_df.iterrows():
    rouge_arrow = "🔼" if row["ROUGE 개선"] > 0 else ("🔽" if row["ROUGE 개선"] < 0 else "➖")
    emb_arrow = "🔼" if row["임베딩 개선"] > 0 else ("🔽" if row["임베딩 개선"] < 0 else "➖")
    print(f"{row['Task']:12s} | ROUGE {rouge_arrow} {row['ROUGE 개선']:+.3f} | 임베딩 {emb_arrow} {row['임베딩 개선']:+.3f}")

## 14. 허깅페이스에 업로드하기 (선택)

학습한 LoRA 어댑터를 허깅페이스에 업로드하여 공유하거나 7번 노트북에서 사용할 수 있습니다.

### 📋 준비 사항

1. **허깅페이스 계정**: https://huggingface.co 회원가입
2. **Access Token 발급**:
   - https://huggingface.co/settings/tokens
   - `New token` → `Write` 권한 선택
   - 토큰 복사

In [ ]:
# Hugging Face 로그인
from huggingface_hub import login

# HF 토큰 입력 (https://huggingface.co/settings/tokens)
login(token="YOUR_HF_TOKEN_HERE")

# 감정 분류 모델 업로드
repo_name_sentiment = "your-username/lora-sentiment"  # 원하는 이름으로 변경
model_sent.push_to_hub(repo_name_sentiment)
tokenizer_sent.push_to_hub(repo_name_sentiment)
print(f"✅ 감정 분류 업로드 완료: https://huggingface.co/{repo_name_sentiment}")

# 영어 QA 모델 업로드
repo_name_qa = "your-username/lora-qa"  # 원하는 이름으로 변경
model_qa.push_to_hub(repo_name_qa)
tokenizer_qa.push_to_hub(repo_name_qa)
print(f"✅ 영어 QA 업로드 완료: https://huggingface.co/{repo_name_qa}")

## 15. 인사이트 및 결론

### 🔍 태스크별 분석

#### 1. 감정 분류
- **난이도**: 낮음 (단순 분류)
- **기대 개선**: 높음
- **특징**: "긍정" 또는 "부정" 단어만 출력하면 됨
- **실전 활용**: 고객 리뷰 분석, 소셜 미디어 모니터링

#### 2. 영어 QA
- **난이도**: 중간 (정보 추출)
- **기대 개선**: 중간
- **특징**: 컨텍스트에서 정확한 정보를 찾아야 함
- **실전 활용**: 문서 검색, FAQ 자동 응답

### 📊 성능 패턴

| 현상 | 설명 | 대응 방법 |
|------|------|-----------|
| **분류 태스크가 쉬움** | 출력이 단순함 (긍정/부정) | 적은 데이터로도 학습 가능 |
| **QA 태스크는 중간** | 정보 추출 능력 필요 | 더 많은 컨텍스트 예제 필요 |
| **태스크별 차이** | 데이터 형식이 다름 | 프롬프트 최적화 중요 |

### ✅ 성공 기준

- **ROUGE 개선 > 0.05**: 의미 있는 향상
- **임베딩 개선 > 0.03**: 의미적 유사도 향상
- **모든 태스크 개선**: LoRA가 일반적으로 효과적

### 🚀 다음 단계

1. **실습 7**: Gradio로 배포하여 실제 사용자 테스트
2. **하이퍼파라미터 튜닝**: 
   - 데이터 증가 (200 → 1000)
   - Epoch 증가 (3 → 10)
   - LoRA rank 증가 (r=8 → r=16)
3. **프롬프트 최적화**: 각 태스크에 맞는 최적 프롬프트 찾기
4. **한국어 요약 추가**: 03번에서 학습한 모델과 함께 3개 모델 비교

### 💡 핵심 인사이트

1. **동일 모델, 다른 데이터 = 다른 결과**
   - LoRA는 태스크별로 독립적으로 학습됨
   - 각 어댑터는 특정 태스크에만 특화

2. **작은 데이터로도 효과 확인 가능**
   - 200개 샘플로도 개선 확인
   - 실전에서는 1000개 이상 권장

3. **태스크 난이도가 다름**
   - 단순 분류: 빠르게 학습
   - 복잡한 생성: 더 많은 데이터 필요

축하합니다! Day 1의 실습 6을 완료했습니다! 🎉

**다음**: 실습 7에서 이 모델들을 Gradio로 배포해봅시다!